# Einarbeitung in Python-Chess
   - Eröffnungsbibliothek
   - Endspiel-Bibliothek: Syzygy
   - Version 0: zufällige Züge auswählen im Mittelspiel

WICHTIG: Alles soll als Jupyter-Notebook implementiert werden.  
Jede Funktion muss ausführlich dokumentiert werden (mittleres Markdown + LaTeX), d.h.
- a) Welche Argumente erhält die Funktion.
- b) Welches Ergebnis wird berechnet.
- c) Welche Seiteneffekte treten auf (falls Seiteneffekte auftreten).
- d) Bei komplizierter Logik soll auch Algorithmus beschrieben werden.

Testen:
- Abspeichern einer Partie als Datei auf Festplatte in algebraischer Notation.
- Verschiedene Versionen Ihrer KIs sollen gegeneinander antreten können.
- Nur Bibliotheken verwenden, die unter Windows lauffähig sind.

Bei Verwendung von Random immer Seed setzen.

In [ ]:
#!pip install chess
import chess
import chess.polyglot
import chess.syzygy
from IPython.display import clear_output, display
from typing import Union, Tuple
import random
from time import sleep

Zunächst definieren wir einen Enum, um durch den folgenden Programmcode hinweg eine bessere Lesbarkeit zu gewährleisten.

In [ ]:
from enum import Enum

class State(Enum):
    OPENING_GAME = 0
    MIDDLE_GAME = 1
    END_GAME = 2
    FINISHED = 3

Diese Funktion wird aufgerufen, solange die gegebene Partie sich im Middlegame befindet und gibt einen zufällig ausgewählten Zug aus der Menge aller gültigen Züge, sowie den Middlegame Status zurück.

Falls der vorherige Status dem des Eröffnungsspiels entspricht, wird der `opening_game` reader der polyglot library geschlossen, da das Eröffnungsspiel hiermit vorüber ist.

In [ ]:
def get_next_middle_game_move() -> chess.Move:
    '''Picks a random middle game move out of all legal board moves'''
    moves = list(board.legal_moves)
    move = moves[random.randint(0, len(moves) - 1)]
    return move

TBD @piuswalter

In [ ]:
def get_next_end_game_move() -> chess.Move:
    moves = []
    for move in board.legal_moves:
        board.push(move)
        wdl = tablebase.get_wdl(board)
        dtz = tablebase.get_dtz(board)
        board.pop()
        moves.append((move, wdl, dtz))
    best_wdl = max(moves, key=lambda move: move[1])[1]
    moves = [move for move in moves if move[1] == best_wdl]
    best_move = min(moves, key=lambda move: move[2])[0]
    return best_move

Diese Funktion gibt den nächsten Spielzug, sowie den neuen Spielstatus zurück.

- Falls der aktuelle Spielstatus dem des Eröffnungsspiel entspricht, wird ein neuer Spielzug aus der polyglot Eröffnungsbibliothek zurückgegeben.
- Falls der Status **nicht** dem des Eröffnungsspiels entspricht, wird überprüft, ob die Endspiel Bibliothek bereits einen Spielzug für die aktuelle Spielsituation findet.
  - Falls ja, wird der nächste Spielzug und Status aus der dazugehörigen Hilfsfunktion zurückgegeben.
  - Sollte die Endspielbibliothek **keinen** gültigen Zug finden, wird überprüft, ob auf dem Spielbrett noch gültige Spielzüge verfügbar sind.
    - Ist dies **nicht** der Fall, so wird das Spiel beendet.
    - Sind gültige Züge vorhanden, so wird die Hilfsfunktion für das Mittelspiel aufgerufen.

In [ ]:
def get_next_move(state: State) -> Tuple[Union[chess.Move, None], State]:
    '''Figure out the next possible move and switch between different game states if necessary'''   
    # abort when fifty-move rule applies
    if board.can_claim_fifty_moves():
        return None, State.FINISHED
    # if there are moves in the opening library find and return the best move
    elif state == State.OPENING_GAME and list(opening_game.find_all(board)):
        return opening_game.find(board).move, State.OPENING_GAME
    # if the endgame library provides a move use it
    elif tablebase.get_wdl(board) != None:
        return get_next_end_game_move(), State.END_GAME
    # if neither the opening nor the endgame library has any moves available choose randomly
    elif list(board.legal_moves):
        return get_next_middle_game_move(), State.MIDDLE_GAME
    # if there are no moves at all return none and end game
    else:
        return None, State.FINISHED

Hier wird ein neues Spiel initialisiert und ein fester Seed definiert. Der statische Seed dient dazu, sämtliche Zufallsfunktionen reproduzierbar zu machen. Der Spielzustand wird initial auf `OPENING_GAME` gesetzt.

Im Anschluss wird die `get_next_move()` Funktion in Dauerschleife so lange ausgeführt, bis sie keinen Spielzug mehr zurückliefert.

> WARNUNG: Bei falscher Implementierung bleibt der Kernel in der Dauerschleife gefangen und muss manuell beendet werden. 

In [ ]:
# Create a new board
board = chess.Board()

# Set a seed, so randomization is reproducable
random.seed(1)

# Start with opening game
state = State.OPENING_GAME

opening_game = chess.polyglot.open_reader("opening/Human-polyglot/Human.bin")

with chess.syzygy.open_tablebase("ending/3-4-5") as tablebase:
    while True:
        next_move, next_state = get_next_move(state)
        if next_move:
            board.push(next_move)
        else:
            print(f"Finished with: {state} -> {next_state}")
            break
        clear_output(wait=True)
        display(board)
        if next_state != state:
            print(f"\n Transition from {state} to {next_state} \n")
            input("Press enter to continue")
            if state == State.OPENING_GAME:
                opening_game.close()
            state = next_state